In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "2021-03-21")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "./common_functions"

In [0]:
%run "./configuration"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import to_timestamp, col, concat, lit

In [0]:
races_schema = StructType(fields=[
    StructField("raceId", IntegerType(), False),
    StructField("year", IntegerType(), False),
    StructField("round", IntegerType(), False),
    StructField("circuitId", IntegerType(), False),
    StructField("name", StringType(), True),
    StructField("date", DateType(), False),
    StructField("time", StringType(), False),
    StructField("url", StringType(), True),
])

In [0]:
races_df = spark.read.options(header=True).schema(races_schema).csv(f"{raw_folder_path}/{v_file_date}/races.csv")

In [0]:
races_transformed_df = races_df \
    .withColumn("race_timestamp", to_timestamp(concat(col("date"), lit(" "), col("time")), "yyyy-MM-dd HH:mm:ss"))

In [0]:
races_transformed_df = add_ingestion_date(races_transformed_df)

In [0]:
races_renamed_df = races_transformed_df \
    .withColumnRenamed("circuitId", "circuit_id") \
    .withColumnRenamed("year", "race_year") \
    .withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("alt", "altitude") \
    .withColumn("data_source", lit(v_data_source)) \
    .withColumn("file_date", lit(v_file_date)) \
    .drop("date", "time", "url")

In [0]:
display(races_renamed_df.printSchema())

root
 |-- race_id: integer (nullable = true)
 |-- race_year: integer (nullable = true)
 |-- round: integer (nullable = true)
 |-- circuit_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- race_timestamp: timestamp (nullable = true)
 |-- ingestion_date: timestamp (nullable = false)
 |-- data_source: string (nullable = false)
 |-- file_date: string (nullable = false)



In [0]:
races_renamed_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed.races")

In [0]:
dbutils.notebook.exit("Success")